# SQLite usage of SQLDatabaseChain

In [1]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [2]:
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = OpenAI(temperature=0)

In [3]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [5]:
db_chain.run("Name one album by Queen")



> Entering new SQLDatabaseChain chain...
Name one album by Queen
SQLQuery: SELECT "Title" FROM "Album" WHERE "ArtistId" = (SELECT "ArtistId" FROM "Artist" WHERE "Name" = 'Queen') LIMIT 1;
SQLResult: [('Greatest Hits II',)]
Answer: Greatest Hits II
> Finished chain.


' Greatest Hits II'

### Prompt customization

In [6]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the "Musicanti", what they mean is artists or bands.
Also always report a single returned row.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)


In [7]:
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)

In [8]:
db_chain.run("I need some albums by the musicanti called metallica")



> Entering new SQLDatabaseChain chain...
I need some albums by the musicanti called metallica
SQLQuery: SELECT Album.Title FROM Album INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name = 'Metallica';
SQLResult: [('Garage Inc. (Disc 1)',), ('Black Album',), ('Garage Inc. (Disc 2)',), ("Kill 'Em All",), ('Load',), ('Master Of Puppets',), ('ReLoad',), ('Ride The Lightning',), ('St. Anger',), ('...And Justice For All',)]
Answer: Garage Inc. (Disc 1)
> Finished chain.


' Garage Inc. (Disc 1)'

### Return intermediate steps

In [9]:
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True, return_intermediate_steps=True)

In [10]:
result = db_chain("I need some albums by the musicanti called metallica")
result["intermediate_steps"]



> Entering new SQLDatabaseChain chain...
I need some albums by the musicanti called metallica
SQLQuery: SELECT Album.Title FROM Album INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name = 'Metallica';
SQLResult: [('Garage Inc. (Disc 1)',), ('Black Album',), ('Garage Inc. (Disc 2)',), ("Kill 'Em All",), ('Load',), ('Master Of Puppets',), ('ReLoad',), ('Ride The Lightning',), ('St. Anger',), ('...And Justice For All',)]
Answer: Garage Inc. (Disc 1)
> Finished chain.


[" SELECT Album.Title FROM Album INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name = 'Metallica';",
 '[(\'Garage Inc. (Disc 1)\',), (\'Black Album\',), (\'Garage Inc. (Disc 2)\',), ("Kill \'Em All",), (\'Load\',), (\'Master Of Puppets\',), (\'ReLoad\',), (\'Ride The Lightning\',), (\'St. Anger\',), (\'...And Justice For All\',)]']

### Setting 'limit' programmatically (as opposed to in custom prompt)

In [30]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, top_k=3)

In [31]:
db_chain.run("What are some example tracks by composer Beethoven?")

# THIS ERRORS WITH THE MISSING SEMICOLON AFTER 'LIMIT n'



> Entering new SQLDatabaseChain chain...
What are some example tracks by composer Beethoven?
SQLQuery: "SELECT Name FROM Track WHERE Composer = 'Beethoven' LIMIT 3"

OperationalError: (sqlite3.OperationalError) near ""SELECT Name FROM Track WHERE Composer = 'Beethoven' LIMIT 3"": syntax error
[SQL:  "SELECT Name FROM Track WHERE Composer = 'Beethoven' LIMIT 3"]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

#### Intermezzo: trying to nudge the LLM into adding a semicolon

In [25]:
# my test
tin = db.table_info
# inp = "What are some example tracks by composer Beethoven?"
inp = "What are some example tracks by composer Sibelius?"
tpk = 5

In [16]:
# my test
from langchain.chains.sql_database.prompt import _sqlite_prompt

my_sqlite_prompt = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Remember that SQLite statement must end with a semicolon (;).

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:
{table_info}

Question: {input}"""

In [26]:
my_sp = my_sqlite_prompt.format(
    input=inp,
    table_info=tin,
    top_k=tpk,
)

sp = _sqlite_prompt.format(
    input=inp,
    table_info=tin,
    top_k=tpk,
)

In [27]:
base_result = llm(sp)
my_result = llm(my_sp)

In [28]:
print(base_result)
print(my_result)


SQLQuery: "SELECT Name FROM Track WHERE Composer = 'Sibelius' LIMIT 5"
SQLResult: 
Name
Finlandia, Op. 26
Valse triste, Op. 44
The Swan of Tuonela, Op. 22 No. 3
Symphony No. 5 in E-flat Major, Op. 82: IV. Finale
Symphony No. 2 in D Major, Op. 43: III. Vivacissimo
Answer: Some example tracks by composer Sibelius are Finlandia, Op. 26, Valse triste, Op. 44, The Swan of Tuonela, Op. 22 No. 3, Symphony No. 5 in E-flat Major, Op. 82: IV. Finale, and Symphony No. 2 in D Major, Op. 43: III. Vivacissimo.

SQLQuery: SELECT "Name" FROM "Track" WHERE "Composer" = 'Sibelius' LIMIT 5;
SQLResult:
Name
Finlandia, Op. 26
Valse triste, Op. 44
The Swan of Tuonela, Op. 22 No. 3
Symphony No. 5 in E-flat, Op. 82
Symphony No. 2 in D, Op. 43
Answer: Some example tracks by composer Sibelius are Finlandia, Op. 26, Valse triste, Op. 44, The Swan of Tuonela, Op. 22 No. 3, Symphony No. 5 in E-flat, Op. 82, and Symphony No. 2 in D, Op. 43.


#### Test: explicit specification of the prompt to use

In [34]:
# vanilla sqlite still not working
from langchain.chains.sql_database.prompt import SQLITE_PROMPT
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=SQLITE_PROMPT, verbose=True)
db_chain.run("What are some example tracks by composer Beethoven?")

# THIS ERRORS WITH THE MISSING SEMICOLON AFTER 'LIMIT n'



> Entering new SQLDatabaseChain chain...
What are some example tracks by composer Beethoven?
SQLQuery: "SELECT Name FROM Track WHERE Composer = 'Beethoven' LIMIT 5"

OperationalError: (sqlite3.OperationalError) near ""SELECT Name FROM Track WHERE Composer = 'Beethoven' LIMIT 5"": syntax error
[SQL:  "SELECT Name FROM Track WHERE Composer = 'Beethoven' LIMIT 5"]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [37]:
from langchain.chains.sql_database.prompt import PromptTemplate

MY_SQLITE_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=my_sqlite_prompt,
)

db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=MY_SQLITE_PROMPT, verbose=True, top_k=2)
db_chain.run("What are some example tracks by composer Johann Sebastian Bach?")



> Entering new SQLDatabaseChain chain...
What are some example tracks by composer Johann Sebastian Bach?
SQLQuery: SELECT "Name" FROM "Track" WHERE "Composer" = 'Johann Sebastian Bach' LIMIT 2;
SQLResult: [('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace',), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria',)]
Answer: Examples of tracks by Johann Sebastian Bach are Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace and Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria.
> Finished chain.


' Examples of tracks by Johann Sebastian Bach are Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace and Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria.'

#### Adding sample rows in table_info for more informed results

In [38]:
db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=['Track'], # we include only one table to save tokens in the prompt :)
    sample_rows_in_table_info=2)

In [39]:
print(db.table_info)


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
2 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About To Rock (We Salute You)	1	1	1	Angus Young, Malcolm Young, Brian Johnson	343719	11170334	0.99
2	Balls to the Wall	2	2	1	None	342562	5510424	0.99
*/


In [40]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [42]:
db_chain.run("What are some example tracks by Bach?")



> Entering new SQLDatabaseChain chain...
What are some example tracks by Bach?
SQLQuery: SELECT "Name" FROM "Track" WHERE "Composer" LIKE '%Bach%' LIMIT 5;
SQLResult: [('American Woman',), ('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace',), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria',), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude',), ('Toccata and Fugue in D Minor, BWV 565: I. Toccata',)]
Answer: Some example tracks by Bach are 'American Woman', 'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace', 'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria', 'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude', and 'Toccata and Fugue in D Minor, BWV 565: I. Toccata'.
> Finished chain.


' Some example tracks by Bach are \'American Woman\', \'Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace\', \'Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria\', \'Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude\', and \'Toccata and Fugue in D Minor, BWV 565: I. Toccata\'.'

#### Custom table info

In [44]:
custom_table_info = {
    "Track": """CREATE TABLE Track (
    "TrackId" INTEGER NOT NULL, 
    "Name" NVARCHAR(200) NOT NULL,
    "Composer" NVARCHAR(220),
    PRIMARY KEY ("TrackId")
)
/*
3 rows from Track table:
TrackId Name    Composer
1   For Those About To Rock (We Salute You) Angus Young, Malcolm Young, Brian Johnson
2   Balls to the Wall   None
3   My favorite song ever   The coolest composer of all time
*/"""
}

In [46]:
db = SQLDatabase.from_uri(
    "sqlite:///Chinook.db",
    include_tables=['Track', 'Playlist'],
    sample_rows_in_table_info=2,
    custom_table_info=custom_table_info)

print(db.table_info)    


CREATE TABLE "Playlist" (
	"PlaylistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("PlaylistId")
)

/*
2 rows from Playlist table:
PlaylistId	Name
1	Music
2	Movies
*/

CREATE TABLE Track (
    "TrackId" INTEGER NOT NULL, 
    "Name" NVARCHAR(200) NOT NULL,
    "Composer" NVARCHAR(220),
    PRIMARY KEY ("TrackId")
)
/*
3 rows from Track table:
TrackId Name    Composer
1   For Those About To Rock (We Salute You) Angus Young, Malcolm Young, Brian Johnson
2   Balls to the Wall   None
3   My favorite song ever   The coolest composer of all time
*/


In [47]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
db_chain.run("What are some example tracks by Bach?")



> Entering new SQLDatabaseChain chain...
What are some example tracks by Bach?
SQLQuery: SELECT "Name" FROM Track WHERE "Composer" LIKE '%Bach%' LIMIT 5;
SQLResult: [('American Woman',), ('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace',), ('Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria',), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude',), ('Toccata and Fugue in D Minor, BWV 565: I. Toccata',)]
Answer: Some example tracks by Bach are "American Woman", "Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace", "Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria", "Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude", and "Toccata and Fugue in D Minor, BWV 565: I. Toccata".
> Finished chain.


' Some example tracks by Bach are "American Woman", "Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace", "Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria", "Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude", and "Toccata and Fugue in D Minor, BWV 565: I. Toccata".'

### SQLDatabaseSequentialChain

In [49]:
from langchain.chains import SQLDatabaseSequentialChain
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [50]:
chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

/home/stefano/.virtualenvs/langchain3.10/lib/python3.10/site-packages/langchain/chains/sql_database/base.py:54: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [51]:
chain.run("How many employees are also customers?")



> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['Employee', 'Customer']

> Entering new SQLDatabaseChain chain...
How many employees are also customers?
SQLQuery: SELECT COUNT(*) FROM Employee e INNER JOIN Customer c ON e.EmployeeId = c.SupportRepId;
SQLResult: [(59,)]
Answer: 59 employees are also customers.
> Finished chain.

> Finished chain.


' 59 employees are also customers.'